In [1]:
import heyoka as hy
import numpy as np

In [5]:
x, y, vx, vy = hy.make_vars("x", "y", "vx", "vy")

eqns = [(x, vx),
        (y, vy),
        (vx, hy.expression(0.)),
        (vy, hy.expression(-9.8))]

eq_w_curve = y - (1. - x + 0.05 * hy.cos(11 * np.pi * x))
eq_bottom = y

Cr = .8

last_t = 0.
bounce_points = []

def cb_w_curve(ta, mr):
    global last_t

    if ta.time - last_t < 1e-10:
        return False

    last_t = ta.time

    x, y = ta.state[0:2]
    bounce_points.append((x, y))
    
    # Compute the normal unit vector
    # using the gradient of the event
    # equation.
    grad = np.array([1+0.05*11*np.pi*np.sin(11*np.pi*x), 1])
    grad_uvec = grad / np.linalg.norm(grad)
    
    # Compute the component of the velocity
    # across the normal vector.
    xy_vel = ta.state[2:4]
    vproj = np.dot(xy_vel, grad_uvec)
    
    # Flip it.
    Dv = -vproj*grad_uvec
    xy_vel += (1. + Cr) * Dv

    return True

def cb_bottom(ta, mr):
    global last_t

    if ta.time - last_t < 1e-10:
        return False

    last_t = ta.time

    x, y = ta.state[0:2]
    bounce_points.append((x, y))
    
    ta.state[3] = -Cr*ta.state[3]
    
    return True

ta = hy.taylor_adaptive(eqns, [0, 1.2, 0, 0], t_events = [hy.t_event(eq_w_curve, callback = cb_w_curve),
                                                          hy.t_event(eq_bottom, callback = cb_bottom)])

In [6]:
t_grid = np.linspace(0, 10, 10000)
oc, _1, _2, _3, res = ta.propagate_grid(t_grid)

b_points = np.array(bounce_points)

In [7]:
%matplotlib widget
from matplotlib.pylab import plt

fig = plt.figure()
plt.plot(res[:,0],res[:,1])
x_grid = np.linspace(0, 1., 1000)
plt.plot(x_grid, (1 - x_grid + 0.05 * np.cos(11 * np.pi * x_grid)), 'k-', linewidth=1)
plt.plot(b_points[:, 0], b_points[:, 1], 'o', alpha=.5)
plt.fill_between(x_grid, -1., (1 - x_grid + 0.05 * np.cos(11 * np.pi * x_grid)), color='gray',)
plt.ylim(0, None);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …